In [ ]:
import os
import glob
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import patches as mpl_patches
import itertools
import pandas as pd
import matplotlib.patheffects as PathEffects
%matplotlib inline
import seaborn as sns

# pyspark libs
#import pyspark
#from pyspark import SparkContext 
#from pyspark.mllib.clustering import BisectingKMeans, BisectingKMeansModel

In [ ]:
## directory where your put lib_hdfs
libdir='/home/rubywerman/scratch-midway2/lib_hdfs'

sys.path.insert(1,os.path.join(sys.path[0],libdir)) # this line helps you to use your own functions in another directory
from alignment_lib import _gen_patches
from alignment_lib import const_clouds_array
from alignment_lib import gen_mod02_img_sigle,  gen_mod35_img_single
from alignment_lib import mod02_proc_sds_single
from alignment_lib import _gen_patches
from alignment_lib import const_clouds_array

In [ ]:
homedir = libdir
datadir = homedir+"/model/m2_02_global_2000_2018_band28_29_31"
step = 100000 # DONOT change so far

encoder_def = datadir+'/encoder.json'
encoder_weight = datadir+'/encoder-'+str(step)+'.h5'
with open(encoder_def, "r") as f:
    encoder = tf.keras.models.model_from_json(f.read())
encoder.load_weights(encoder_weight)

In [ ]:
%reload_ext autoreload
%autoreload 2
from pyhdf.SD import SD, SDC

# directory where your put lib_hdfs
# e.g. /home/tkurihana/src/src_share/lib_hdfs
homedir='/home/rubywerman/scratch-midway2/lib_hdfs'

In [4]:
sys.path.insert(1,os.path.join(sys.path[0],homedir)) # this line helps you to use your own functinos in another directory
from alignment_lib import gen_mod02_img_sigle,  gen_mod35_img_single
from alignment_lib import mod02_proc_sds_single
from alignment_lib import _gen_patches
from alignment_lib import const_clouds_array


# enter name of text file containing the dates of the closed patches
closed_file_name = "/home/rubywerman/clouds/src_analysis/close_cells_mod02/close_dates.txt"
closed_text_file = open(closed_file_name, "r")
closed_dates = closed_text_file.read().split('\n')

# enter name of text file containing the dates of the open patches
open_file_name = "/home/rubywerman/clouds/src_analysis/open_cells_mod02/open_dates.txt"
open_text_file = open(open_file_name, "r")
open_dates = open_text_file.read().split('\n')

#select directory path for given date
def get_data(d):
    return (homedir+'/mod02/open_chile_3/' + d, homedir+'/mod35/open_chile_3/' + d, get_coded_date(d))

W0712 16:35:03.127253 140297144956736 deprecation.py:506] From /home/rubywerman/.conda/envs/clouds/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0712 16:35:03.129428 140297144956736 deprecation.py:506] From /home/rubywerman/.conda/envs/clouds/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/home/rubywerman/.conda/envs/clouds/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:891: UserWarning: models is not loaded, but a Lambda layer uses it.

In [ ]:
class Patch:        
    def __init__(self, date, isOpen, thirtyFive, zeroTwo, label=None, feature=None, has_coord=False, coords=None):
        self.date = date
        self.isOpen = isOpen
        self.thirtyFive = thirtyFive
        self.zeroTwo = zeroTwo
        self.label = label
        self.feature = feature
        self.has_coord = has_coord
        self.coords = coords
        
    def print_attr(self):
        print("date: " + self.date)
        print("isOpen: " + str(self.isOpen))
        print("label: " + str(self.label))
        if len(self.coords) > 0:
            print("coords: ")
            for i in self.coords:
                print(str(i))
        
        
class Cluster:
    def __init__(self, label, patches=None, means=None, std=None, num_open=0):
        self.label = label
        self.patches = patches
        self.means = means
        self.std = std
        self.num_open = num_open

In [5]:
#generates list of 6 means for each band in the patch list 
def get_band_means(patch_list):
    mean_list = []
    for i in range(6):
        mean_list.append(np.nanmean([patch.zeroTwo[0,0,:,:,:][:,:,i] for patch in patch_list]))
    return mean_list

#generates list of 6 stds for each band in the patch list 
def get_band_std(patch_list):
    std_list = []
    for i in range(6):
        std_list.append(np.nanstd([patch.zeroTwo[0,0,:,:,:][:,:,i] for patch in patch_list]))
    return std_list

#collect close patch 35 and 02 data
closed_patch_class_list = []
#closed_clouds_mask_img_data = []
for d in closed_dates:
    if d:
        file_name_a = '/home/rubywerman/clouds/src_analysis/close_cells_mod02/closed_'+ d +'.npy'
        #closed_date_data = np.load(file_name_a[:72] + file_name_a[-4:])
        closed_date_data = np.load(file_name_a)
        #file_name_b = '/home/rubywerman/clouds/src_analysis/close_cells_coords/closed_coords'+ d +'.npy'
        #thirtyFive = np.load('/home/rubywerman/clouds/src_analysis/close_35/close_35_'+ d +'.npy')
        #coords = np.load(file_name_b[:79] + file_name_b[-4:])
        #ccloud_img = np.load('/home/rubywerman/clouds/src_analysis/close_cells_mod35/closed_35_'+ d +'.npy')
        #closed_clouds_mask_img_data.append(ccloud_img)
        for patch in range(len(closed_date_data)):
            closed_patch_class_list.append(Patch(date=d, isOpen=False, thirtyFive=None, zeroTwo=closed_date_data[patch], label=None, feature=None, has_coord=True, coords=None))
#collect open patch 35 and 02 data
#open_clouds_mask_img_data = []
open_patch_class_list = []
for d in open_dates:
    if d:
        file_name_a = '/home/rubywerman/clouds/src_analysis/open_cells_mod02/open_'+ d +'.npy'
        #open_date_data = np.load(file_name_a[:69] + file_name_a[-4:])
        open_date_data = np.load(file_name_a)
        #file_name_b = '/home/rubywerman/clouds/src_analysis/open_cells_coords/open_coords'+ d +'.npy'
        #coords = np.load(file_name_b[:76] + file_name_b[-4:])
        #thirtyFive = np.load('/home/rubywerman/clouds/src_analysis/open_35/open_35_'+ d +'.npy')
        #ccloud_img = np.load('/home/rubywerman/clouds/src_analysis/close_cells_mod35/closed_35_'+ d +'.npy')
        #closed_clouds_mask_img_data.append(ccloud_img)
        for patch in range(len(open_date_data)):
            open_patch_class_list.append(Patch(date=d, isOpen=True, thirtyFive=None, zeroTwo=open_date_data[patch], label=None, feature=None, has_coord=True, coords=None))
# here make patches list
class_patch_list = open_patch_class_list + closed_patch_class_list
lenn = len(open_patch_class_list)
print("Number of patches recorded: " + str(len(class_patch_list)))            
            

Number of patches recorded: 239


In [ ]:
encs_list = []
for patch in class_patch_list:
    i = patch.zeroTwo
    if type(i) is not list:
        ix, iy = i.shape[:2]
        encs = encoder.predict(i.reshape(ix * iy, 128,128,6))
        encs_list += [encs.mean(axis=(1,2))]
features = np.concatenate(encs_list, axis=0)
print(features.shape)  # make sure, the shape is [#number of patches, 128]
#the first %num_open_features rows in features are open cells 
num_open_features = len(open_patch_class_list)
#starting at row %num_open_features in features they are closed cells 
num_closed_features = len(closed_patch_class_list)
from sklearn.cluster import AgglomerativeClustering
num_clusters = 2
clustering = AgglomerativeClustering(num_clusters)
#turns any NAN values to 0 so code doesn't crash
cleaned_features = np.nan_to_num(features)
#generate clustering data
label = clustering.fit_predict(cleaned_features)
open_labels = label[:num_closed_features]
closed_labels = label[num_closed_features:]
#assign each patch instance its correct label
for i in range(len(class_patch_list)):
    class_patch_list[i].label = label[i]
    
#make a list of clusters
list_clusters = [Cluster(label=i) for i in range(num_clusters)]
#add patches to cluster objects
for p in class_patch_list:
    matching_cluster = list_clusters[p.label]
    if p.isOpen:
        matching_cluster.num_open += 1
    if not matching_cluster.patches:
        matching_cluster.patches = [p]
    else:
        matching_cluster.patches.append(p)
#add patch mean and std to cluster objects
for c in list_clusters:
    c.means = get_band_means(c.patches)
    c.std = get_band_std(c.patches)

In [ ]:
#generate number of open and closed patches in each cluster
open_clusters = [i.num_open for i in list_clusters]
closed_clusters = [len(i.patches) - i.num_open for i in list_clusters]

In [ ]:
open_in_0 = [patch for patch in list_clusters[0].patches if patch.isOpen]
o_list_pics = []
for i in range(len(open_in_0)):
    p = open_in_0[i]
    o_list_pics.append(p.thirtyFive[p.coords[0]*128:p.coords[1]*128,p.coords[2]*128:p.coords[3]*128])
len(o_list_pics)

In [ ]:
closed_in_0 = [patch for patch in list_clusters[0].patches if not patch.isOpen]
c_list_pics = []
for i in range(len(closed_in_0)):
    p = closed_in_0[i]
    c_list_pics.append(p.thirtyFive[p.coords[0]*128:p.coords[1]*128,p.coords[2]*128:p.coords[3]*128])
len(c_list_pics)

In [ ]:
w=16
h=16
fig1=plt.figure(figsize=(8, 8))
columns1 = len(o_list_pics)
rows1 = 1 
for i in range(1, columns1*rows1 +1):
    fig1.add_subplot(rows1, columns1, i)
    plt.imshow(o_list_pics[i-1])
plt.show()

In [ ]:
w=16
h=16
fig=plt.figure(figsize=(8, 8))
columns = len(c_list_pics)//8
rows = columns + 1
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(c_list_pics[i-1])
plt.show()